In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import pandas as pd

books = pd.read_csv(r"C:\Users\harip\Desktop\Semantic_Book_recommender\data\data_processed.csv")

In [4]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9.780002e+12,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883.0 A NOVEL THAT READERS and criti...
1,9.780002e+12,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982.0 A new 'Christie for Christmas'...
2,9.780006e+12,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736.0 A memorable, mesmerizing heroi..."
3,9.780006e+12,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897.0 Lewis' work on the nature of l...
4,9.780006e+12,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934.0 ""In The Problem of Pain, C.S. ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9.788172e+12,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222.0 On A Train Journey Home To Nor...
5193,9.788173e+12,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014.0 This book tells the tale of a ...
5194,9.788180e+12,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623.0 Wisdom to Create a Life of Pas...
5195,9.788185e+12,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535.0 This collection of the timeles...


In [5]:
books["tagged_description"]


0       9780002005883.0 A NOVEL THAT READERS and criti...
1       9780002261982.0 A new 'Christie for Christmas'...
2       9780006178736.0 A memorable, mesmerizing heroi...
3       9780006280897.0 Lewis' work on the nature of l...
4       9780006280934.0 "In The Problem of Pain, C.S. ...
                              ...                        
5192    9788172235222.0 On A Train Journey Home To Nor...
5193    9788173031014.0 This book tells the tale of a ...
5194    9788179921623.0 Wisdom to Create a Life of Pas...
5195    9788185300535.0 This collection of the timeles...
5196    9789027712059.0 Since the three volume edition...
Name: tagged_description, Length: 5197, dtype: object

In [6]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [7]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1170, which is longer than the specified 100
Created a chunk of size 1216, which is longer than the specified 100
Created a chunk of size 375, which is longer than the specified 100
Created a chunk of size 311, which is longer than the specified 100
Created a chunk of size 485, which is longer than the specified 100
Created a chunk of size 484, which is longer than the specified 100
Created a chunk of size 962, which is longer than the specified 100
Created a chunk of size 190, which is longer than the specified 100
Created a chunk of size 845, which is longer than the specified 100
Created a chunk of size 298, which is longer than the specified 100
Created a chunk of size 199, which is longer than the specified 100
Created a chunk of size 883, which is longer than the specified 100
Created a chunk of size 1090, which is longer than the specified 100
Created a chunk of size 1191, which is longer than the specified 100
Created a chunk of size 306, which is longer

In [8]:
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings())

In [9]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='b18e87be-a1c6-4d41-a397-070b5b6ccab2', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069.0 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='644607d0-0f33-4b0a-a03e-05e4cfe2b841', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380.0 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='c1d86db9-291c-4b96-9c19-ff2110847e8e', metadata={'source

In [11]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip().replace('.0', ''))]


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9.780787e+12,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069.0 Children will discover the exc...


In [15]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0].replace('.0', ''))]

    return books[books["isbn13"].isin(books_list)]

In [16]:
retrieve_semantic_recommendations("A book to teach children about nature")


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
31,9.780007e+12,0007105045,Tree and Leaf,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,Tree and Leaf: The Homecoming of Beorhtnoth : ...,"9780007105045.0 ""The two works 'On fairy-stori..."
59,9.780007e+12,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240.0 It should be the most natural ...
143,9.780061e+12,0060546573,Three Rotten Eggs,Gregory Maguire,Juvenile Fiction,http://books.google.com/books/content?id=t2pWl...,The students of Miss Earth's class in rural Ve...,2005.0,3.74,240.0,76.0,Three Rotten Eggs,9780060546571.0 The students of Miss Earth's c...
324,9.780061e+12,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036.0 Barbara Kingsolver's fifth nov...
429,9.780064e+12,0064434982,The Deer in the Wood,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=V7YDW...,Even the youngest child can enjoy a special ad...,1999.0,4.17,32.0,302.0,The Deer in the Wood,9780064434980.0 Even the youngest child can en...
442,9.780068e+12,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208.0 First published more than thre...
692,9.780140e+12,0140448004,Three Tales,Gustave Flaubert;Roger Whitehouse;Geoffrey Wall,Fiction,http://books.google.com/books/content?id=XFzga...,Features short fiction by the French naturalis...,2005.0,3.71,110.0,3050.0,Three Tales,9780140448009.0 Features short fiction by the ...
707,9.780141e+12,0140568190,The Giraffe and the Pelly and Me,Roald Dahl;Quentin Blake,Candy,http://books.google.com/books/content?id=J7FdI...,"A Dahl story in which the giraffe, the pelican...",2001.0,3.81,32.0,16265.0,The Giraffe and the Pelly and Me,9780140568196.0 A Dahl story in which the gira...
711,9.780141e+12,0140621628,The Railway Children,E. Nesbit,Fiction,http://books.google.com/books/content?id=fFesd...,"When their father is sent away to prison, thre...",1995.0,4.00,212.0,178.0,The Railway Children,9780140621624.0 When their father is sent away...
763,9.780141e+12,0141186070,The Log from the Sea of Cortez,John Steinbeck,Biography & Autobiography,http://books.google.com/books/content?id=9CrIf...,This light-hearted journal tells of John Stein...,2001.0,3.84,288.0,3226.0,The Log from the Sea of Cortez,9780141186078.0 This light-hearted journal tel...
